In [1]:
import numpy as np
import pandas as pd
from mne_bids import BIDSPath, read_raw_bids, get_entity_vals
import mne
from ptsa.data.timeseries import TimeSeries
np.set_printoptions(edgeitems=2, threshold=10)
from pathlib import Path
from typing import Iterable, Any
import re
from typing import Sequence
import numpy as np
import pandas as pd
import warnings
from bidsreader import BIDSReader

Needs:
* Load eeg and ieeg for cml
* Set dataset collection fodlder, study root, subject, task, session and not deal with mne_bids
* get events both eeg/ieeg and behavioral
* get all subjects in a study
* get all tasks in a study
* get all tasks for a subject in a study
* get all sessions for a subject [and task] in a study
* get scalp eeg data
* convert from eeg to ptsa (merge events) have to update to mne_bids 0.18 on the new server

* get electrodes
* get monopolar df
* get bipolar df
* return pairs and contacts (combined versions of dfs with electrodes)
* get monopolar edf
* get bipolar bdf
* get epochs
* get events and event_id
* get space


2 classes
1: BIDSReader (for each study)
2: DatasetCollectionReader (keeps track of all studies in the folder)

In [2]:
# root
dataset_collection_root = Path("/data/LTP_BIDS")
study_name = "FR1"

study_root = dataset_collection_root / study_name

# Specify which subject and experiment we want
sub = 'R1111M'
task = 'FR1'
ses = 0

In [3]:
# tests
bidsreader = BIDSReader(root=study_root)
bidsreader.get_dataset_subjects()
bidsreader.get_dataset_tasks()
bidsreader.get_dataset_max_sessions(outlier_thresh=50)

10

In [4]:
bidsreader = BIDSReader(root=study_root, subject=sub)
bidsreader.task = task
bidsreader.session = ses
bidsreader.eeg_type = 'ieeg'
evs = bidsreader.load_events("beh")

In [5]:
bidsreader = BIDSReader(root=study_root, subject=sub)
bidsreader.task = task
bidsreader.session = ses
bidsreader.eeg_type = 'ieeg'
bidsreader.space
bidsreader.load_electrodes()

,name,x,y,z,size,group,hemisphere,type,tal.x,tal.y,tal.z,wb.region,ind.region,stein.region
0,LPOG1,-67.9554,-20.436300,-26.318920,-999,LPOG,L,grid,-66.7592,-20.37470,-21.06940,NaN,middletemporal,NaN
1,LPOG2,-71.3723,-19.887300,-17.033223,-999,LPOG,L,grid,-68.5270,-19.30560,-13.11050,NaN,middletemporal,NaN
2,LPOG3,-69.4694,-16.873900,-5.837007,-999,LPOG,L,grid,-67.0028,-17.99460,-3.26183,NaN,middletemporal,NaN
3,LPOG4,-68.4177,-13.619100,6.599195,-999,LPOG,L,grid,-62.8222,-17.49650,6.35027,NaN,superiortemporal,NaN
4,LPOG5,-68.5695,-14.288000,16.220750,-999,LPOG,L,grid,-60.2654,-16.09750,16.38480,NaN,postcentral,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LPS4,-60.3150,0.724204,-33.789406,-999,LPS,L,strip,-59.5567,-3.04176,-28.33260,NaN,middletemporal,NaN
96,LTD1,-24.4314,-20.139100,-23.616084,-999,LTD,L,depth,-23.8723,-23.65220,-19.49340,Left PHG parahippocampal gyrus,parahippocampal,Left EC
97,LTD2,-28.6866,-18.553800,-24.592941,-999,LTD,L,depth,-28.2112,-20.97920,-19.44620,Left Cerebral White Matter,parahippocampal,Left MTL WM
98,LTD3,-33.2518,-18.690900,-25.448912,-999,LTD,L,depth,-33.5565,-19.32430,-18.68380,Left PHG parahippocampal gyrus,parahippocampal,Left PRC


In [6]:
bidsreader.load_channels("bipolar")

,name,type,units,low_cutoff,high_cutoff,reference,group,sampling_frequency,description,notch
0,LPOG1-LPOG9,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN
1,LPOG1-LPOG2,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN
2,LPOG2-LPOG10,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN
3,LPOG2-LPOG3,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN
4,LPOG3-LPOG4,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN
...,...,...,...,...,...,...,...,...,...,...
136,LPS2-LPS3,ECOG,V,NaN,NaN,bipolar,LPS,500,strip,NaN
137,LPS3-LPS4,ECOG,V,NaN,NaN,bipolar,LPS,500,strip,NaN
138,LTD1-LTD2,SEEG,V,NaN,NaN,bipolar,LTD,500,depth,NaN
139,LTD2-LTD3,SEEG,V,NaN,NaN,bipolar,LTD,500,depth,NaN


In [7]:
bidsreader.load_channels("monopolar")

,name,type,units,low_cutoff,high_cutoff,group,sampling_frequency,description,notch
0,LPOG1,ECOG,V,NaN,NaN,LPOG,500,grid,NaN
1,LPOG2,ECOG,V,NaN,NaN,LPOG,500,grid,NaN
2,LPOG3,ECOG,V,NaN,NaN,LPOG,500,grid,NaN
3,LPOG4,ECOG,V,NaN,NaN,LPOG,500,grid,NaN
4,LPOG5,ECOG,V,NaN,NaN,LPOG,500,grid,NaN
...,...,...,...,...,...,...,...,...,...
95,LPS4,ECOG,V,NaN,NaN,LPS,500,strip,NaN
96,LTD1,SEEG,V,NaN,NaN,LTD,500,depth,NaN
97,LTD2,SEEG,V,NaN,NaN,LTD,500,depth,NaN
98,LTD3,SEEG,V,NaN,NaN,LTD,500,depth,NaN


In [8]:
bidsreader.load_combined_channels("bipolar")

,name,type,units,low_cutoff,high_cutoff,reference,group,sampling_frequency,description,notch,...,tal.z_ch2,wb.region_pair,ind.region_pair,stein.region_pair,x_mid,y_mid,z_mid,tal.x_mid,tal.y_mid,tal.z_mid
0,LPOG1-LPOG9,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN,...,-20.14920,NaN,middletemporal,NaN,-66.68055,-24.669050,-26.717305,-66.60470,-25.48000,-20.609300
1,LPOG1-LPOG2,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN,...,-13.11050,NaN,middletemporal,NaN,-69.66385,-20.161800,-21.676071,-67.64310,-19.84015,-17.089950
2,LPOG2-LPOG10,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN,...,-10.86980,NaN,middletemporal,NaN,-70.96495,-24.684950,-16.724423,-68.34015,-24.39265,-11.990150
3,LPOG2-LPOG3,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN,...,-3.26183,NaN,middletemporal,NaN,-70.42085,-18.380600,-11.435115,-67.76490,-18.65010,-8.186165
4,LPOG3-LPOG4,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN,...,6.35027,NaN,NaN,NaN,-68.94355,-15.246500,0.381094,-64.91250,-17.74555,1.544220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,LPS2-LPS3,ECOG,V,NaN,NaN,bipolar,LPS,500,strip,NaN,...,-36.38240,NaN,NaN,NaN,-49.30180,-1.581440,-46.198862,-49.88840,-4.68133,-38.805100
137,LPS3-LPS4,ECOG,V,NaN,NaN,bipolar,LPS,500,strip,NaN,...,-28.33260,NaN,middletemporal,NaN,-57.54940,-0.408223,-38.124577,-57.07530,-3.48274,-32.357500
138,LTD1-LTD2,SEEG,V,NaN,NaN,bipolar,LTD,500,depth,NaN,...,-19.44620,NaN,parahippocampal,NaN,-26.55900,-19.346450,-24.104513,-26.04175,-22.31570,-19.469800
139,LTD2-LTD3,SEEG,V,NaN,NaN,bipolar,LTD,500,depth,NaN,...,-18.68380,NaN,parahippocampal,NaN,-30.96920,-18.622350,-25.020927,-30.88385,-20.15175,-19.065000


In [9]:
bidsreader.load_combined_channels("monopolar")

,name,type,units,low_cutoff,high_cutoff,group,sampling_frequency,description,notch,x,...,size,group_elec,hemisphere,type_elec,tal.x,tal.y,tal.z,wb.region,ind.region,stein.region
0,LPOG1,ECOG,V,NaN,NaN,LPOG,500,grid,NaN,-67.9554,...,-999,LPOG,L,grid,-66.7592,-20.37470,-21.06940,NaN,middletemporal,NaN
1,LPOG2,ECOG,V,NaN,NaN,LPOG,500,grid,NaN,-71.3723,...,-999,LPOG,L,grid,-68.5270,-19.30560,-13.11050,NaN,middletemporal,NaN
2,LPOG3,ECOG,V,NaN,NaN,LPOG,500,grid,NaN,-69.4694,...,-999,LPOG,L,grid,-67.0028,-17.99460,-3.26183,NaN,middletemporal,NaN
3,LPOG4,ECOG,V,NaN,NaN,LPOG,500,grid,NaN,-68.4177,...,-999,LPOG,L,grid,-62.8222,-17.49650,6.35027,NaN,superiortemporal,NaN
4,LPOG5,ECOG,V,NaN,NaN,LPOG,500,grid,NaN,-68.5695,...,-999,LPOG,L,grid,-60.2654,-16.09750,16.38480,NaN,postcentral,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LPS4,ECOG,V,NaN,NaN,LPS,500,strip,NaN,-60.3150,...,-999,LPS,L,strip,-59.5567,-3.04176,-28.33260,NaN,middletemporal,NaN
96,LTD1,SEEG,V,NaN,NaN,LTD,500,depth,NaN,-24.4314,...,-999,LTD,L,depth,-23.8723,-23.65220,-19.49340,Left PHG parahippocampal gyrus,parahippocampal,Left EC
97,LTD2,SEEG,V,NaN,NaN,LTD,500,depth,NaN,-28.6866,...,-999,LTD,L,depth,-28.2112,-20.97920,-19.44620,Left Cerebral White Matter,parahippocampal,Left MTL WM
98,LTD3,SEEG,V,NaN,NaN,LTD,500,depth,NaN,-33.2518,...,-999,LTD,L,depth,-33.5565,-19.32430,-18.68380,Left PHG parahippocampal gyrus,parahippocampal,Left PRC


In [10]:
raw = bidsreader.load_raw(acquisition="monopolar")

Extracting EDF parameters from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-monopolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_events.tsv.
Reading channel info from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-monopolar_channels.tsv.
Reading electrode coords from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_space-MNI152NLin6ASym_electrodes.tsv.


/home1/zrentala/bidsreader/bidsreader.py:450: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  raw = read_raw_bids(bp)


In [11]:
bidsreader.load_raw(acquisition="bipolar")

Extracting EDF parameters from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-bipolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_events.tsv.
Reading channel info from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-bipolar_channels.tsv.
Reading electrode coords from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_space-MNI152NLin6ASym_electrodes.tsv.


/home1/zrentala/bidsreader/bidsreader.py:450: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  raw = read_raw_bids(bp)
/home1/zrentala/bidsreader/bidsreader.py:309: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage, on_missing="ignore")


<RawEDF | sub-R1111M_ses-0_task-FR1_acq-bipolar_ieeg.edf, 141 x 1623500 (3247.0 s), ~162 kB, data not loaded>

In [12]:
bidsreader.load_epochs(tmin=0, tmax=1.6, trial_types=["WORD"], baseline=None, acquisition="bipolar", event_repeated="merge",preload=False)

Extracting EDF parameters from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-bipolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_events.tsv.
Reading channel info from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-bipolar_channels.tsv.
Reading electrode coords from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_space-MNI152NLin6ASym_electrodes.tsv.
Used Annotations descriptions: ['COUNTDOWN_END', 'COUNTDOWN_START', 'DISTRACT_END', 'DISTRACT_START', 'ORIENT', 'PRACTICE_DISTRACT_END', 'PRACTICE_DISTRACT_START', 'PRACTICE_REC_END', 'PRACTICE_REC_START', 'PRACTICE_WORD', 'PROB', 'REC_END', 'REC_START', 'REC_WORD', 'SESS_START', 'START', 'STOP', 'TRIAL', 'WORD']
Used Annotations descriptions: ['COUNTDOWN_END', 'COUNTDOWN_START', 'DISTRACT_END', 'DISTRACT_START', 'ORIENT', 'PRACTICE_DISTR

/home1/zrentala/bidsreader/bidsreader.py:450: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  raw = read_raw_bids(bp)
/home1/zrentala/bidsreader/bidsreader.py:309: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage, on_missing="ignore")


Number of events,288
Events,WORD: 288
Time range,0.000 – 1.600 s
Baseline,off


In [13]:
epoch = bidsreader.load_epochs(tmin=0, tmax=1.6, trial_types=None, baseline=None, acquisition="bipolar", event_repeated="merge", preload=False)

Extracting EDF parameters from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-bipolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_events.tsv.
Reading channel info from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-bipolar_channels.tsv.
Reading electrode coords from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_space-MNI152NLin6ASym_electrodes.tsv.
Used Annotations descriptions: ['COUNTDOWN_END', 'COUNTDOWN_START', 'DISTRACT_END', 'DISTRACT_START', 'ORIENT', 'PRACTICE_DISTRACT_END', 'PRACTICE_DISTRACT_START', 'PRACTICE_REC_END', 'PRACTICE_REC_START', 'PRACTICE_WORD', 'PROB', 'REC_END', 'REC_START', 'REC_WORD', 'SESS_START', 'START', 'STOP', 'TRIAL', 'WORD']
Multiple event values for single event times found. Creating new event value to reflect simultaneous events.
Not setting metada

/home1/zrentala/bidsreader/bidsreader.py:450: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  raw = read_raw_bids(bp)
/home1/zrentala/bidsreader/bidsreader.py:309: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage, on_missing="ignore")


In [14]:
epoch

Number of events,722
Events,COUNTDOWN_END: 25COUNTDOWN_START: 2COUNTDOWN_START/TRIAL: 24DISTRACT_END: 24DISTRACT_START: 7DISTRACT_START/START: 17ORIENT: 24PRACTICE_DISTRACT_END: 1PRACTICE_DISTRACT_START/START: 1PRACTICE_REC_END: 1PRACTICE_REC_START: 1PRACTICE_WORD: 12PROB: 82REC_END: 24REC_START: 24REC_WORD: 131SESS_START: 1START: 7STOP: 25TRIAL: 1WORD: 288
Time range,0.000 – 1.600 s
Baseline,off


In [15]:
bidsreader.load_epochs(tmin=0, tmax=1.6, trial_types=None, baseline=None, acquisition="monopolar", event_repeated="merge", preload=False)

Extracting EDF parameters from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-monopolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_events.tsv.
Reading channel info from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-monopolar_channels.tsv.
Reading electrode coords from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_space-MNI152NLin6ASym_electrodes.tsv.
Used Annotations descriptions: ['COUNTDOWN_END', 'COUNTDOWN_START', 'DISTRACT_END', 'DISTRACT_START', 'ORIENT', 'PRACTICE_DISTRACT_END', 'PRACTICE_DISTRACT_START', 'PRACTICE_REC_END', 'PRACTICE_REC_START', 'PRACTICE_WORD', 'PROB', 'REC_END', 'REC_START', 'REC_WORD', 'SESS_START', 'START', 'STOP', 'TRIAL', 'WORD']
Multiple event values for single event times found. Creating new event value to reflect simultaneous events.
Not setting me

/home1/zrentala/bidsreader/bidsreader.py:450: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  raw = read_raw_bids(bp)


Number of events,722
Events,COUNTDOWN_END: 25COUNTDOWN_START: 2COUNTDOWN_START/TRIAL: 24DISTRACT_END: 24DISTRACT_START: 7DISTRACT_START/START: 17ORIENT: 24PRACTICE_DISTRACT_END: 1PRACTICE_DISTRACT_START/START: 1PRACTICE_REC_END: 1PRACTICE_REC_START: 1PRACTICE_WORD: 12PROB: 82REC_END: 24REC_START: 24REC_WORD: 131SESS_START: 1START: 7STOP: 25TRIAL: 1WORD: 288
Time range,0.000 – 1.600 s
Baseline,off


In [16]:
BIDSReader.mne_to_ptsa(epoch, evs)

Loading data for 722 events and 801 original time points ...
0 bad epochs dropped


<xarray.TimeSeries (event: 722, channel: 141, time: 801)>
array([[[ 0.0062969 ,  0.00628596, ...,  0.00622853,  0.00622238],
        [ 0.00712763,  0.0071119 , ...,  0.00698541,  0.00698951],
        ...,
        [-0.010159  , -0.01015383, ..., -0.01020273, -0.01020394],
        [-0.00149898, -0.00150293, ..., -0.00144492, -0.00144796]],

       [[ 0.00640972,  0.00639673, ...,  0.00615948,  0.0061417 ],
        [ 0.00717344,  0.00715839, ...,  0.00714267,  0.00714198],
        ...,
        [-0.01031782, -0.01031388, ..., -0.01032785, -0.01032694],
        [-0.00143672, -0.00143672, ..., -0.00140393, -0.00140271]],

       ...,

       [[ 0.00627297,  0.00625246, ...,  0.00651433,  0.00650886],
        [ 0.00718643,  0.0071707 , ...,  0.0074565 ,  0.00745581],
        ...,
        [-0.01028685, -0.01029171, ..., -0.01017782, -0.01017296],
        [-0.00141273, -0.00141486, ..., -0.00140696, -0.00140787]],

       [[ 0.00621759,  0.00623674, ...,  0.00639878,  0.00640972],
        [ 0.0071707 ,  0.00716591, ...,  0.00723976,  0.00723839],
        ...,
        [-0.01028199, -0.01027895, ..., -0.01030294, -0.01030598],
        [-0.0013769 , -0.00138176, ..., -0.00135199, -0.00134896]]])
Coordinates: (12/18)
  * event          (event) object MultiIndex
  * onset          (event) float64 0.0 18.21 28.64 ... 3.122e+03 9.039e+04
  * duration       (event) float64 18.21 10.0 3.709 1.6 ... 15.27 282.5 0.0 0.0
  * sample         (event) int64 23207 32311 37524 39379 ... 1442760 1583983 -1
  * trial_type     (event) object 'SESS_START' ... 'SESSION_SKIPPED'
  * response_time  (event) float64 nan nan nan nan nan ... 14.75 nan nan nan
    ...             ...
  * experiment     (event) object 'FR1' 'FR1' 'FR1' 'FR1' ... 'FR1' 'FR1' 'FR1'
  * session        (event) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
  * subject        (event) object 'R1111M' 'R1111M' ... 'R1111M' 'R1111M'
  * channel        (channel) <U13 'LPOG1-LPOG9' 'LPOG1-LPOG2' ... 'LTD3-LTD4'
  * time           (time) float64 0.0 0.002 0.004 0.006 ... 1.596 1.598 1.6
    samplerate     float64 500.0

In [17]:

filtered_df, evs_raw, e_id = BIDSReader.filter_events_by_trial_types(["WORD", "REC_WORD"], events_df=evs, raw=raw)

Used Annotations descriptions: ['COUNTDOWN_END', 'COUNTDOWN_START', 'DISTRACT_END', 'DISTRACT_START', 'ORIENT', 'PRACTICE_DISTRACT_END', 'PRACTICE_DISTRACT_START', 'PRACTICE_REC_END', 'PRACTICE_REC_START', 'PRACTICE_WORD', 'PROB', 'REC_END', 'REC_START', 'REC_WORD', 'SESS_START', 'START', 'STOP', 'TRIAL', 'WORD']


In [18]:
filtered_df

,onset,duration,sample,trial_type,response_time,stim_file,item_name,serialpos,list,test,answer,experiment,session,subject
27,154.635,1.600,100520,WORD,NaN,wordpools/wordpool_EN.txt,BEAR,1,1,"[0, 0, 0]",NaN,FR1,0,R1111M
28,157.252,1.600,101829,WORD,NaN,wordpools/wordpool_EN.txt,WING,2,1,"[0, 0, 0]",NaN,FR1,0,R1111M
29,159.820,1.600,103113,WORD,NaN,wordpools/wordpool_EN.txt,DOOR,3,1,"[0, 0, 0]",NaN,FR1,0,R1111M
30,162.253,1.600,104329,WORD,NaN,wordpools/wordpool_EN.txt,PLANT,4,1,"[0, 0, 0]",NaN,FR1,0,R1111M
31,164.871,1.600,105638,WORD,NaN,wordpools/wordpool_EN.txt,ROOT,5,1,"[0, 0, 0]",NaN,FR1,0,R1111M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,2811.736,1.263,1428997,REC_WORD,2.487,NaN,RICE,-999,24,"[0, 0, 0]",NaN,FR1,0,R1111M
757,2812.999,1.095,1429628,REC_WORD,3.750,NaN,SOAP,-999,24,"[0, 0, 0]",NaN,FR1,0,R1111M
758,2814.094,4.294,1430176,REC_WORD,4.845,NaN,SEA,-999,24,"[0, 0, 0]",NaN,FR1,0,R1111M
759,2818.388,5.608,1432322,REC_WORD,9.139,NaN,SHIELD,-999,24,"[0, 0, 0]",NaN,FR1,0,R1111M


In [19]:
# get recall
key = ['list','item_name']

w_df = filtered_df.copy()
w_df['recalled'] = False

df_enc = w_df.loc[w_df.trial_type == 'WORD'][key]
df_rec = w_df.loc[w_df.trial_type == 'REC_WORD'][key]

recalled = df_enc.merge(df_rec,on=key,how='inner')

if not recalled.empty:
    idx = pd.MultiIndex.from_frame(recalled)
    df_idx = pd.MultiIndex.from_frame(w_df[key])
    mask = df_idx.isin(idx) & (w_df['trial_type'] == 'WORD').values
    w_df.loc[mask, 'recalled'] = True

In [20]:
w_df.query('trial_type == "WORD" and list == 1')

,onset,duration,sample,trial_type,response_time,stim_file,item_name,serialpos,list,test,answer,experiment,session,subject,recalled
27,154.635,1.6,100520,WORD,NaN,wordpools/wordpool_EN.txt,BEAR,1,1,"[0, 0, 0]",NaN,FR1,0,R1111M,True
28,157.252,1.6,101829,WORD,NaN,wordpools/wordpool_EN.txt,WING,2,1,"[0, 0, 0]",NaN,FR1,0,R1111M,True
29,159.820,1.6,103113,WORD,NaN,wordpools/wordpool_EN.txt,DOOR,3,1,"[0, 0, 0]",NaN,FR1,0,R1111M,True
30,162.253,1.6,104329,WORD,NaN,wordpools/wordpool_EN.txt,PLANT,4,1,"[0, 0, 0]",NaN,FR1,0,R1111M,True
31,164.871,1.6,105638,WORD,NaN,wordpools/wordpool_EN.txt,ROOT,5,1,"[0, 0, 0]",NaN,FR1,0,R1111M,False
32,167.388,1.6,106897,WORD,NaN,wordpools/wordpool_EN.txt,LEAF,6,1,"[0, 0, 0]",NaN,FR1,0,R1111M,False
33,169.805,1.6,108105,WORD,NaN,wordpools/wordpool_EN.txt,SNOW,7,1,"[0, 0, 0]",NaN,FR1,0,R1111M,False
34,172.339,1.6,109372,WORD,NaN,wordpools/wordpool_EN.txt,BLOOM,8,1,"[0, 0, 0]",NaN,FR1,0,R1111M,False
35,174.756,1.6,110580,WORD,NaN,wordpools/wordpool_EN.txt,STRAW,9,1,"[0, 0, 0]",NaN,FR1,0,R1111M,False
36,177.222,1.6,111813,WORD,NaN,wordpools/wordpool_EN.txt,BUSH,10,1,"[0, 0, 0]",NaN,FR1,0,R1111M,False


In [21]:
evs_raw

array([[  77318,       0,      19],
       [  78626,       0,      19],
       ...,
       [1409194,       0,      14],
       [1411998,       0,      14]])

In [22]:
e_id

{'REC_WORD': 14, 'WORD': 19}

In [23]:
BIDSReader.raw_to_ptsa(raw)

<xarray.TimeSeries (channel: 100, time: 1623500)>
array([[ 3.59565462e-03,  3.59485769e-03, ...,  3.05658132e-07,
         3.05658132e-07],
       [-3.53168738e-03, -3.51694418e-03, ...,  3.05658132e-07,
         3.05658132e-07],
       ...,
       [ 1.63596635e-03,  1.63193617e-03, ..., -6.39667959e-08,
        -6.39667959e-08],
       [ 3.13497046e-03,  3.13497046e-03, ..., -6.39667959e-08,
        -6.39667959e-08]])
Coordinates:
  * channel     (channel) object 'LPOG1' 'LPOG2' 'LPOG3' ... 'LTD3' 'LTD4'
  * time        (time) float64 0.0 0.002 0.004 ... 3.247e+03 3.247e+03 3.247e+03
    samplerate  float64 500.0
Attributes:
    mne_meas_date:   2024-04-04 18:13:27+00:00
    mne_first_samp:  0